# 1. Load / Ingest the data using sqoop and hive in Clourera VM

## 1.1. Check the list of tables in MySQL database with sqoop list-tables command

In [21]:
#Listing all tables in db_bdtt_ac database
! sqoop list-tables \
--connect jdbc:mysql://localhost/db_bdtt_ac --username training --password training

19/04/21 10:08:48 INFO sqoop.Sqoop: Running Sqoop version: 1.4.5-cdh5.4.3
19/04/21 10:08:48 WARN tool.BaseSqoopTool: Setting your password on the command-line is insecure. Consider using -P instead.
19/04/21 10:08:49 INFO manager.MySQLManager: Preparing to use a MySQL streaming resultset.
accident_severity
accidents
casualties


## 1.2. Import Accidents table from MySQL to HDFS

Here, I have imported accidents table from MySQL to hadoop file system by using credentials from database.props file.I have used warehouse directory to create a folder with table name

In [6]:
! sqoop --options-file ./database.props \
        --table accidents \
        --split-by 'Accident_Index'\
        --warehouse-dir /db_bdtt_ac \
        --null-string '-1' \
        --null-non-string '-1' \
        --m 1

19/04/21 08:29:24 INFO sqoop.Sqoop: Running Sqoop version: 1.4.5-cdh5.4.3
19/04/21 08:29:24 WARN tool.BaseSqoopTool: Setting your password on the command-line is insecure. Consider using -P instead.
19/04/21 08:29:25 INFO manager.MySQLManager: Preparing to use a MySQL streaming resultset.
19/04/21 08:29:25 INFO tool.CodeGenTool: Beginning code generation
19/04/21 08:29:25 INFO manager.SqlManager: Executing SQL statement: SELECT t.* FROM `accidents` AS t LIMIT 1
19/04/21 08:29:25 INFO manager.SqlManager: Executing SQL statement: SELECT t.* FROM `accidents` AS t LIMIT 1
19/04/21 08:29:25 INFO orm.CompilationManager: HADOOP_MAPRED_HOME is /usr/lib/hadoop-mapreduce
Note: /tmp/sqoop-training/compile/9b8fcf75d6abd7288fe06622c4c2645b/accidents.java uses or overrides a deprecated API.
Note: Recompile with -Xlint:deprecation for details.
19/04/21 08:29:29 INFO orm.CompilationManager: Writing jar file: /tmp/sqoop-training/compile/9b8fcf75d6abd7288fe06622c4c2645b/accidents.jar
19/04/21 08:29:29 W

#### 1.2.1 Check the warehouse directory for import status

In [7]:
! hdfs dfs -ls /db_bdtt_ac/accidents

Found 2 items
-rw-rw-rw-   1 training supergroup          0 2019-04-21 08:29 /db_bdtt_ac/accidents/_SUCCESS
-rw-rw-rw-   1 training supergroup   19004812 2019-04-21 08:29 /db_bdtt_ac/accidents/part-m-00000


####1.2.2. Cheking imported data with default number of mappers in warehouse directory

In [9]:
! hdfs dfs -cat /db_bdtt_ac/accidents/part-m-00000 | head

2017010001708,532920.0,196330.0,-0.08010700,51.65006100,1,1,2,3,05/08/2017,7,03:12,32,E09000010,3,105,6,30.0,0,-1,-1,0,0,0,4,1,1,0,0,1,1,E01001450
2017010009342,526790.0,181970.0,-0.17384500,51.52242500,1,3,2,1,01/01/2017,1,01:30,1,E09000033,3,5,6,30.0,3,4,6,0,0,0,4,1,2,0,0,1,1,E01004702
2017010009344,535200.0,181260.0,-0.05296900,51.51409600,1,3,3,1,01/01/2017,1,00:30,5,E09000030,3,13,6,30.0,3,4,5,0,0,0,4,1,1,0,0,1,1,E01004298
2017010009348,534340.0,193560.0,-0.06065800,51.62483200,1,3,2,1,01/01/2017,1,01:11,32,E09000010,3,1010,1,30.0,1,4,4,154,0,4,4,2,2,0,0,1,1,E01001429
2017010009350,533680.0,187820.0,-0.07237200,51.57340800,1,2,1,1,01/01/2017,1,01:42,4,E09000012,3,107,3,20.0,6,2,3,10,0,5,4,1,2,0,0,1,1,E01001808
2017010009351,514510.0,172370.0,-0.35387600,51.43876200,1,3,2,1,01/01/2017,1,03:31,24,E09000027,6,0,6,30.0,0,-1,-1,0,0,0,4,1,2,0,0,1,1,E01003900
2017010009353,508640.0,181870.0,-0.43537700,51.52530500,1,3,2,2,01/01/2017,1,04:07,26,E09000017,3,4020,3,40.0,3,4,5,0,0,4,4,1,2,0,

## 1.3. Import Accidents table from HDFS into Hive

In [13]:
%%sh 
hive -e "CREATE EXTERNAL TABLE if not exists db_bdtt_ac.accidents(Accident_Index varchar(50),Location_Easting_OSGR FLOAT,Location_Northing_OSGR FLOAT,Longitude DECIMAL(10,8),Latitude DECIMAL(10,8),Police_Force INT,Accident_Severity INT,Number_of_Vehicles INT,Number_of_Casualties INT,Date VARCHAR(50),Day_of_Week INT,Time VARCHAR(50),Local_Authority_District INT,Local_Authority_Highway VARCHAR(50), st_Road_Class INT,1st_Road_Number INT,Road_Type INT,Speed_limit FLOAT,Junction_Detail INT,Junction_Control INT,2nd_Road_Class INT,2nd_Road_Number INT,Pedestrian_Crossing_Human_Control INT,Pedestrian_Crossing_Physical_Facilities INT,Light_Conditions INT,Weather_Conditions INT,Road_Surface_Conditions INT,Special_Conditions_at_Site INT,Carriageway_Hazards INT,Urban_or_Rural_Area INT,Did_Police_Officer_Attend_Scene_of_Accident INT,LSOA_of_Accident_Location VARCHAR(50)) comment 'External table for accidents data' row format delimited fields terminated by ',' location '/db_bdtt_ac/accidents/' tblproperties(\"skip.header.line.count\"=\"1\")"


Logging initialized using configuration in file:/etc/hive/conf.dist/hive-log4j.properties
OK
Time taken: 1.063 seconds


In [14]:
%%sh
hive -e "select * from db_bdtt_ac.accidents limit 10"

2017010009342	526790.0	181970.0	-0.173845	51.522425	1	3	2	1	01/01/2017	1	01:30	1	E09000033	3	5	6	30.0	3	4	6	0	0	0	4	1	2	0	0	1	1	E01004702
2017010009344	535200.0	181260.0	-0.052969	51.514096	1	3	3	1	01/01/2017	1	00:30	5	E09000030	3	13	6	30.0	3	4	5	0	0	0	4	1	1	0	0	1	1	E01004298
2017010009348	534340.0	193560.0	-0.060658	51.624832	1	3	2	1	01/01/2017	1	01:11	32	E09000010	3	1010	1	30.0	1	4	4	154	0	4	4	2	2	0	0	1	1	E01001429
2017010009350	533680.0	187820.0	-0.072372	51.573408	1	2	1	1	01/01/2017	1	01:42	4	E09000012	3	107	3	20.0	6	2	3	10	0	5	4	1	2	0	0	1	1	E01001808
2017010009351	514510.0	172370.0	-0.353876	51.438762	1	3	2	1	01/01/2017	1	03:31	24	E09000027	6	0	6	30.0	0	-1	-1	0	0	0	4	1	2	0	0	1	1	E01003900
2017010009353	508640.0	181870.0	-0.435377	51.525305	1	3	2	2	01/01/2017	1	04:07	26	E09000017	3	4020	3	40.0	3	4	5	0	0	4	4	1	2	0	0	1	1	E01002401
2017010009354	527880.0	181950.0	-0.15815	51.522	1	3	2	1	01/01/2017	1	05:20	1	E09000033	3	501	3	30.0	0	-1	-1	0	2	5	4	2	2	4	0	1	1	E01004660
2017010009357	520


Logging initialized using configuration in file:/etc/hive/conf.dist/hive-log4j.properties
OK
Time taken: 0.981 seconds, Fetched: 10 row(s)


## 1.4. Import casualties table from MySQL into Hive

In [12]:
! sqoop --options-file ./database.props \
        --table casualties \
        --split-by 'Accident_Index'\
        --hive-import \
        --hive-database db_bdtt_ac \
        --hive-table casualties \
        --null-string '-1' \
        --null-non-string '-1' \
        --m 1

19/04/04 00:25:02 INFO sqoop.Sqoop: Running Sqoop version: 1.4.5-cdh5.4.3
19/04/04 00:25:02 WARN tool.BaseSqoopTool: Setting your password on the command-line is insecure. Consider using -P instead.
19/04/04 00:25:02 INFO tool.BaseSqoopTool: Using Hive-specific delimiters for output. You can override
19/04/04 00:25:02 INFO tool.BaseSqoopTool: delimiters with --fields-terminated-by, etc.
19/04/04 00:25:02 INFO manager.MySQLManager: Preparing to use a MySQL streaming resultset.
19/04/04 00:25:02 INFO tool.CodeGenTool: Beginning code generation
19/04/04 00:25:03 INFO manager.SqlManager: Executing SQL statement: SELECT t.* FROM `casualties` AS t LIMIT 1
19/04/04 00:25:03 INFO manager.SqlManager: Executing SQL statement: SELECT t.* FROM `casualties` AS t LIMIT 1
19/04/04 00:25:03 INFO orm.CompilationManager: HADOOP_MAPRED_HOME is /usr/lib/hadoop-mapreduce
Note: /tmp/sqoop-training/compile/ef7a3806099a7f7779be8bff6ff6a5a5/casualties.java uses or overrides a deprecated API.
Note: Recompile wi

In [18]:
%%sh
hive -e "select * from db_bdtt_ac.casualties limit 10"

2017010001708	1	1	2	2	18	4	3	0	0	1	0	0	9	1	2
2017010001708	2	2	1	1	19	4	2	0	0	0	0	0	2	-1	-1
2017010001708	2	3	2	1	18	4	1	0	0	0	0	0	2	-1	-1
2017010009342	1	1	2	2	33	6	3	0	0	1	0	0	9	1	5
2017010009344	3	1	1	2	31	6	3	0	0	0	0	0	9	1	5
2017010009348	1	1	2	1	3	1	3	0	0	2	0	0	9	1	2
2017010009350	1	1	3	1	45	7	2	4	3	0	0	2	0	-1	-1
2017010009351	2	1	1	1	14	3	3	0	0	0	0	0	3	1	9
2017010009353	1	1	1	2	58	9	3	0	0	0	0	0	9	1	2
2017010009353	2	2	1	1	27	6	3	0	0	0	0	0	9	1	3



Logging initialized using configuration in file:/etc/hive/conf.dist/hive-log4j.properties
OK
Time taken: 0.902 seconds, Fetched: 10 row(s)


## 1.5. Import casualties table from MySQL into Hive

In [15]:
%%sh
hive -e "CREATE EXTERNAL TABLE if not exists  db_bdtt_ac.vehicles(Accident_Index VARCHAR(50),Vehicle_Reference INT,Vehicle_Type INT,Towing_and_Articulation INT,Vehicle_Manoeuvre INT,Vehicle_Location_Restricted_Lane INT,Junction_Location INT,Skidding_and_Overturning INT,Hit_Object_in_Carriageway INT,Vehicle_Leaving_Carriageway INT,Hit_Object_off_Carriageway INT,1st_Point_of_Impact INT,Was_Vehicle_Left_Hand_Drive INT,Journey_Purpose_of_Driver INT,Sex_of_Driver INT,Age_of_Driver INT,Age_Band_of_Driver INT,Engine_Capacity_CC INT,Propulsion_Code INT,Age_of_Vehicle INT,Driver_IMD_Decile INT,Driver_Home_Area_Type INT,Vehicle_IMD_Decile INT) comment 'External table for vehicles data' row format delimited fields terminated by ',' location '/db_bdtt_ac/vehicles/' tblproperties(\"skip.header.line.count\"=\"1\");"



Logging initialized using configuration in file:/etc/hive/conf.dist/hive-log4j.properties
OK
Time taken: 0.872 seconds


In [16]:
%%sh
hive -e "select * from db_bdtt_ac.vehicles limit 10"

2017010001708	1	9	0	18	0	0	0	0	0	0	1	1	6	1	24	5	1997	2	1	-1	-1	-1
2017010001708	2	2	0	18	0	0	1	0	0	0	2	1	6	1	19	4	-1	-1	-1	-1	-1	-1
2017010009342	1	9	0	18	0	1	0	0	0	0	2	1	6	1	33	6	1797	8	8	9	1	9
2017010009342	2	9	0	18	0	1	1	0	0	0	1	1	6	1	40	7	2204	2	12	2	1	2
2017010009344	1	9	0	18	0	1	0	0	0	0	1	1	6	3	-1	-1	-1	-1	-1	-1	-1	-1
2017010009344	2	9	0	18	0	2	0	0	0	0	1	1	6	1	35	6	1896	2	9	-1	-1	-1
2017010009344	3	9	0	18	0	2	0	0	0	0	1	1	6	2	31	6	1299	1	14	5	1	5
2017010009348	1	9	0	18	0	4	0	9	0	0	1	1	6	2	37	7	1399	1	9	2	1	2
2017010009348	2	9	0	9	0	3	0	0	0	0	3	1	6	2	29	6	1797	8	6	4	1	4
2017010009350	1	9	0	18	0	2	0	0	0	0	3	1	6	1	78	11	1988	1	14	4	1	4



Logging initialized using configuration in file:/etc/hive/conf.dist/hive-log4j.properties
OK
Time taken: 1.686 seconds, Fetched: 10 row(s)
